<a href="https://colab.research.google.com/github/platypus2000jp/Colaboratory/blob/main/AI%E8%AA%8D%E8%AD%98%E3%81%AETeachable_Machine_%E8%BB%A2%E7%A7%BB%E5%AD%A6%E7%BF%92%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: AI認識のTeachable MachineをPython環境で適当な例題として作成できますか。ただし深層学習のTensorFlow.を転移学習のに置き換えてください。依存関係をインストールしてください。転移学習は再学習できるようにボタンを作り、名前を付けてダウンロードします。

!pip install tensorflow tensorflow-hub numpy matplotlib

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

# Teachable Machineのような、簡単な画像分類モデルを作成します。
# 転移学習を用いて、MobileNetV2をベースに、新しい分類器を構築します。

# 画像データの準備（例として、猫と犬の画像を仮定します）
# 実際のデータは、Google DriveやGCSから読み込むように変更してください。
# 画像は、'cat'と'dog'のフォルダにそれぞれ格納されているとします。
train_data_dir = '/content/train_data'
if not os.path.exists(train_data_dir):
  os.makedirs(train_data_dir)

# 画像の前処理（リサイズなど）とラベル付けを行う関数を定義します。
def preprocess_image(image_path, image_size=(224, 224)):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, image_size)
  image = image / 255.0  # 0～1の範囲に正規化
  return image

# MobileNetV2のモデルをTensorFlow Hubから読み込みます。
module_selection = ("mobilenet_v2", 224, 1280)
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
IMAGE_SIZE = (pixels, pixels)
classifier = tf.keras.Sequential([
    hub.KerasLayer(MODULE_HANDLE, input_shape=IMAGE_SIZE + (3,), trainable=False),
    tf.keras.layers.Dense(2, activation='softmax')  # 2クラス分類（猫と犬）
])

# モデルのコンパイル
classifier.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 学習データの準備
# 画像データを読み込み、ラベルを割り当てます。
# 実際のデータに合わせて変更してください。

train_image_paths = []
train_labels = []

def load_dataset():
    for label_name in os.listdir(train_data_dir):
        label_path = os.path.join(train_data_dir, label_name)
        if os.path.isdir(label_path):
            for image_name in os.listdir(label_path):
                image_path = os.path.join(label_path, image_name)
                train_image_paths.append(image_path)
                if label_name == 'cat':
                    train_labels.append(0)  # 猫をラベル0とする
                else:
                    train_labels.append(1)  # 犬をラベル1とする

load_dataset()

train_images = [preprocess_image(image_path) for image_path in train_image_paths]
train_images = np.array(train_images)
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=2)

# モデルの学習
classifier.fit(train_images, train_labels, epochs=10)

# モデルの保存
classifier.save('my_teaachable_machine_model.h5')

# 再学習用のボタンを作成（例として）
print("モデルをダウンロードして、再学習を行うには、以下の手順に従ってください。")
print("1. 'my_teaachable_machine_model.h5' ファイルをダウンロードします。")
print("2. 新しいデータでモデルを再学習します。")

# ダウンロード用リンクの作成
from google.colab import files
files.download('my_teaachable_machine_model.h5')


ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7fcb3e54f820> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [ ]:
# prompt: 再学習のためにモデルをアップロードして利用します。アップロードしたモデルを使用して、転移学習を継続してください。

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from google.colab import files

# アップロードしたモデルを読み込む
uploaded = files.upload()
model_path = list(uploaded.keys())[0]  # アップロードされたファイル名を取得

# モデルを読み込む
loaded_model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer':hub.KerasLayer})


# 画像データの準備（例として、猫と犬の画像を仮定します）
# 実際のデータは、Google DriveやGCSから読み込むように変更してください。
# 画像は、'cat'と'dog'のフォルダにそれぞれ格納されているとします。
train_data_dir = '/content/train_data'
if not os.path.exists(train_data_dir):
  os.makedirs(train_data_dir)

# 画像の前処理（リサイズなど）とラベル付けを行う関数を定義します。
def preprocess_image(image_path, image_size=(224, 224)):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, image_size)
  image = image / 255.0  # 0～1の範囲に正規化
  return image

# 学習データの準備
# 画像データを読み込み、ラベルを割り当てます。
# 実際のデータに合わせて変更してください。

train_image_paths = []
train_labels = []

def load_dataset():
    for label_name in os.listdir(train_data_dir):
        label_path = os.path.join(train_data_dir, label_name)
        if os.path.isdir(label_path):
            for image_name in os.listdir(label_path):
                image_path = os.path.join(label_path, image_name)
                train_image_paths.append(image_path)
                if label_name == 'cat':
                    train_labels.append(0)  # 猫をラベル0とする
                else:
                    train_labels.append(1)  # 犬をラベル1とする

load_dataset()

train_images = [preprocess_image(image_path) for image_path in train_image_paths]
train_images = np.array(train_images)
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=2)

# モデルのコンパイル（必要に応じて変更）
loaded_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


# 転移学習を継続します。
loaded_model.fit(train_images, train_labels, epochs=10)

# モデルを保存します。
loaded_model.save('retrained_model.h5')

# ダウンロード用リンクの作成
files.download('retrained_model.h5')